# Collinear spin and magnetic systems

In this example we consider iron in the BCC phase.
To show that this material is ferromagnetic we will model it once
allowing collinear spin polarization and once without
and compare the resulting SCF energies. In particular
the ground state can only be found if collinear spins are allowed.

First we setup BCC iron without spin polarization
using a single iron atom inside the unit cell.

In [1]:
using DFTK

a = 5.42352  # Bohr
lattice = a / 2 * [[-1  1  1];
                   [ 1 -1  1];
                   [ 1  1 -1]]
Fe = ElementPsp(:Fe, psp=load_psp("hgh/lda/Fe-q8.hgh"))
atoms = [Fe => [zeros(3)]];

To get the ground-state energy we use an LDA model and rather moderate
discretisation parameters.

In [2]:
kgrid = [3, 3, 3]  # k-point grid (Regular Monkhorst-Pack grid)
Ecut = 15          # kinetic energy cutoff in Hartree
model_nospin = model_LDA(lattice, atoms, temperature=0.01)
basis_nospin = PlaneWaveBasis(model_nospin; kgrid, Ecut)

scfres_nospin = self_consistent_field(basis_nospin, tol=1e-6, mixing=KerkerMixing());

n     Free energy       Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -16.64707741529         NaN   3.28e-01   0.80    5.0
  2   -16.64775238495   -6.75e-04   7.78e-02   0.80    1.0
  3   -16.64782698467   -7.46e-05   3.19e-03   0.80    2.2
  4   -16.64783335296   -6.37e-06   1.54e-03   0.80    3.5
  5   -16.64783428973   -9.37e-07   1.97e-04   0.80    2.0


In [3]:
scfres_nospin.energies

Energy breakdown:
    Kinetic             15.9236257
    AtomicLocal         -5.0708217
    AtomicNonlocal      -5.2218032
    Ewald               -21.4723040
    PspCorrection       1.8758831 
    Hartree             0.7797752 
    Xc                  -3.4439002
    Entropy             -0.0182891

    total               -16.647834289730


Since we did not specify any initial magnetic moment on the iron atom,
DFTK will automatically assume that a calculation with only spin-paired
electrons should be performed. As a result the obtained ground state
features no spin-polarization.

Now we repeat the calculation, but give the iron atom an initial magnetic moment.
For specifying the magnetic moment pass the desired excess of spin-up over spin-down
electrons at each centre to the `Model` and the guess density functions.
In this case we seek the state with as many spin-parallel
``d``-electrons as possible. In our pseudopotential model the 8 valence
electrons are 2 pair of ``s``-electrons, 1 pair of ``d``-electrons
and 4 unpaired ``d``-electrons giving a desired magnetic moment of `4` at the iron centre.
The structure (i.e. pair mapping and order) of the `magnetic_moments` array needs to agree
with the `atoms` array and `0` magnetic moments need to be specified as well.

In [4]:
magnetic_moments = [Fe => [4, ]];

!!! tip "Units of the magnetisation and magnetic moments in DFTK"
    Unlike all other quantities magnetisation and magnetic moments in DFTK
    are given in units of the Bohr magneton ``μ_B``, which in atomic units has the
    value ``\frac{1}{2}``. Since ``μ_B`` is (roughly) the magnetic moment of
    a single electron the advantage is that one can directly think of these
    quantities as the excess of spin-up electrons or spin-up electron density.

We repeat the calculation using the same model as before. DFTK now detects
the non-zero moment and switches to a collinear calculation.

In [5]:
model = model_LDA(lattice, atoms, magnetic_moments=magnetic_moments, temperature=0.01)
basis = PlaneWaveBasis(model; Ecut, kgrid)
ρ0 = guess_density(basis, magnetic_moments)
scfres = self_consistent_field(basis, tol=1e-6; ρ=ρ0, mixing=KerkerMixing());

n     Free energy       Eₙ-Eₙ₋₁     ρout-ρin   Magnet   α      Diag
---   ---------------   ---------   --------   ------   ----   ----
  1   -16.65996073325         NaN   3.11e-01    2.620   0.80    4.5
  2   -16.66626109538   -6.30e-03   6.98e-02    2.428   0.80    1.0
  3   -16.66694859769   -6.88e-04   9.42e-03    2.349   0.80    2.9
  4   -16.66698687690   -3.83e-05   4.34e-03    2.320   0.80    2.9
  5   -16.66699445541   -7.58e-06   1.78e-03    2.309   0.80    2.2
  6   -16.66699630871   -1.85e-06   5.55e-04    2.301   0.80    1.5
  7   -16.66699651926   -2.11e-07   4.49e-05    2.297   0.80    1.5


In [6]:
scfres.energies

Energy breakdown:
    Kinetic             16.3014764
    AtomicLocal         -5.2260055
    AtomicNonlocal      -5.4135560
    Ewald               -21.4723040
    PspCorrection       1.8758831 
    Hartree             0.8201244 
    Xc                  -3.5395371
    Entropy             -0.0130777

    total               -16.666996519264


!!! note "Model and magnetic moments"
    DFTK does not store the `magnetic_moments` inside the `Model`, but only uses them
    to determine the lattice symmetries. This step was taken to keep `Model`
    (which contains the physical model) independent of the details of the numerical details
    such as the initial guess for the spin density.

In direct comparison we notice the first, spin-paired calculation to be
a little higher in energy

In [7]:
println("No magnetization: ", scfres_nospin.energies.total)
println("Magnetic case:    ", scfres.energies.total)
println("Difference:       ", scfres.energies.total - scfres_nospin.energies.total);

No magnetization: -16.647834289729897
Magnetic case:    -16.66699651926364
Difference:       -0.01916222953374458


Notice that with the small cutoffs we use to generate the online
documentation the calculation is far from converged.
With more realistic parameters a larger energy difference of about
0.1 Hartree is obtained.

The spin polarization in the magnetic case is visible if we
consider the occupation of the spin-up and spin-down Kohn-Sham orbitals.
Especially for the ``d``-orbitals these differ rather drastically.
For example for the first ``k``-point:

In [8]:
iup   = 1
idown = iup + length(scfres.basis.kpoints) ÷ 2
@show scfres.occupation[iup][1:7]
@show scfres.occupation[idown][1:7];

(scfres.occupation[iup])[1:7] = [1.0, 0.9999988143551615, 0.9999988143482963, 0.9999988143374605, 0.9585779981428485, 0.9585773615127637, 1.154481319657189e-29]
(scfres.occupation[idown])[1:7] = [1.0, 0.832435779622907, 0.8324334455743521, 0.8324328972842255, 7.827965479462486e-6, 7.827897928579632e-6, 1.5495781414400108e-32]


Similarly the eigenvalues differ

In [9]:
@show scfres.eigenvalues[iup][1:7]
@show scfres.eigenvalues[idown][1:7];

(scfres.eigenvalues[iup])[1:7] = [-0.06926943519825754, 0.3568794267731245, 0.35687948467422975, 0.3568795760651128, 0.4619152650653395, 0.46191542539926445, 1.1596448178304155]
(scfres.eigenvalues[idown])[1:7] = [-0.030402637282866296, 0.477301758611698, 0.47730192594232623, 0.47730196524960666, 0.6109096534575789, 0.6109097397529302, 1.2257790550416954]


!!! note "k-points in collinear calculations"
    For collinear calculations the `kpoints` field of the `PlaneWaveBasis` object contains
    each ``k``-point coordinate twice, once associated with spin-up and once with down-down.
    The list first contains all spin-up ``k``-points and then all spin-down ``k``-points,
    such that `iup` and `idown` index the same ``k``-point, but differing spins.

We can observe the spin-polarization by looking at the density of states (DOS)
around the Fermi level, where the spin-up and spin-down DOS differ.

In [10]:
using Plots
plot_dos(scfres)

Similarly the band structure shows clear differences between both spin components.

Computing bands along kpath:
       Γ -> H -> N -> Γ -> P -> H  and  P -> N
Diagonalising Hamiltonian kblocks: 100%|████████████████| Time: 0:00:03


In [11]:
using Unitful
using UnitfulAtomic
plot_bandstructure(scfres, kline_density=3, unit=u"hartree")